# 问题2：原材料订购与转运方案优化

## 问题分析

问题2要求解决两个核心问题：
1. 确定企业至少需要选择多少家供应商来满足生产需求
2. 制定未来24周最经济的原材料订购方案和损耗最少的转运方案

## 基础数据回顾

基于第一问的分析，我们已经获得：
- 402家供应商的供货特征量化分析
- 50家最重要供应商的排名
- 各供应商的供货稳定性、可靠性评估
- 转运商的损耗率分析

## 约束条件梳理

### 生产需求约束
- 企业每周产能：2.82万立方米
- 原材料消耗比例：
  - A类：0.6立方米/立方米产品
  - B类：0.66立方米/立方米产品  
  - C类：0.72立方米/立方米产品
- 库存要求：不少于满足两周生产需求的库存量

### 供应商约束
- 供应商实际供货量可能偏离订货量
- 需要基于历史数据预测供货稳定性
- 每家供应商有其供应能力上限

### 转运约束
- 每家转运商运输能力：6000立方米/周
- 转运过程中存在损耗（损耗率varies by转运商）
- 一家供应商每周的原材料尽量由一家转运商运输

### 成本约束
- A类原材料采购单价比C类高20%
- B类原材料采购单价比C类高10%
- 运输和储存的单位费用相同

## 解决思路

### 第一阶段：最少供应商数量确定

#### 方法1：基于需求量的理论计算
1. **计算每周最大原材料需求**
   - 按最高效率（A类原材料）：2.82万 × 0.6 = 1.692万立方米/周
   - 按最低效率（C类原材料）：2.82万 × 0.72 = 2.03万立方米/周

2. **考虑安全库存需求**
   - 两周生产需求作为安全库存
   - 总需求 = 周需求 + 安全库存需求

3. **基于TOP50供应商的供应能力**
   - 从第一问结果中提取TOP50供应商的平均供货能力
   - 按供应能力从高到低排序，累计计算直到满足总需求

#### 方法2：基于历史数据的蒙特卡洛模拟
1. **建立供应商供货概率模型**
   - 基于历史数据建立每个供应商的供货量分布
   - 考虑供货的不确定性和波动性

2. **多次模拟验证**
   - 随机选择不同数量的供应商组合
   - 模拟24周的供货情况
   - 计算满足需求的概率，确定最少供应商数量

### 第二阶段：最经济订购方案制定

#### 目标函数设计
$$\text{最小化总成本} = \text{采购成本} + \text{库存成本} + \text{缺货惩罚成本}$$

其中：
- 采购成本 = Σ(订货量 × 单价 × 材料类型系数)
- 库存成本 = Σ(库存量 × 单位库存费用)
- 缺货惩罚成本 = Σ(缺货量 × 单位缺货惩罚)

#### 决策变量
- $x_{i,t}$：第t周向供应商i的订货量
- $I_{t}$：第t周末的库存量
- $S_{t}$：第t周的缺货量

#### 约束条件
1. **需求满足约束**：$\text{库存} + \text{本周接收量} - \text{本周消耗} \geq 0$

2. **库存平衡约束**：$I_t = I_{t-1} + \text{接收量}_t - \text{需求}_t$

3. **最小库存约束**：$I_t \geq 2 \times \text{周需求}$

4. **供应商产能约束**：$x_{i,t} \leq \text{供应商i的最大供应能力}$

5. **非负约束**：$x_{i,t} \geq 0$

### 第三阶段：损耗最少转运方案

#### 目标函数
$$\text{最小化总损耗} = \sum_{i,j,t} (\text{供货量}_{i,t} \times \text{损耗率}_{j,t} \times y_{i,j,t})$$

#### 决策变量
- $y_{i,j,t}$：第t周供应商i是否由转运商j运输（0-1变量）

#### 约束条件
1. **转运商选择约束**：$\sum_j y_{i,j,t} \leq 1$ （每个供应商每周最多选择一个转运商）

2. **转运能力约束**：$\sum_i (\text{供货量}_{i,t} \times y_{i,j,t}) \leq 6000$ （每个转运商的周运输能力）

3. **逻辑约束**：只有当供应商i在第t周有供货时，才能选择转运商

## 求解策略

### 整体求解框架
1. **两阶段优化方法**
   - 第一阶段：固定供应商选择，优化订购量
   - 第二阶段：基于订购方案，优化转运分配

2. **启发式算法**
   - 贪心算法：优先选择性价比高的供应商和转运商
   - 遗传算法：用于供应商组合优化
   - 动态规划：用于时间序列的订购量优化

### 模型验证
1. **历史数据回测**：用前期数据验证模型的有效性
2. **敏感性分析**：分析关键参数变化对结果的影响
3. **鲁棒性测试**：测试方案在不确定环境下的表现

## 预期分析内容

### 订购方案分析
1. **供应商选择结果**：最终选择的供应商数量和名单
2. **订购量分布**：各周、各供应商的具体订购量
3. **成本分析**：总成本构成和每周成本变化
4. **风险评估**：供货风险和库存风险分析

### 转运方案分析
1. **转运商利用率**：各转运商的使用频率和运输量
2. **损耗分析**：总损耗量和损耗率统计
3. **运输效率**：运输能力利用率分析

### 实施效果评估
1. **需求满足率**：各周需求满足程度
2. **库存水平**：库存量变化趋势和安全性
3. **经济效益**：成本节约和效率提升
4. **风险控制**：供应风险的有效控制程度

## 技术实施方案

### 数据预处理
1. **利用第一问的分析结果**
   - 导入供应商可靠性评估数据
   - 导入转运商损耗率分析数据
   - 导入TOP50重要供应商名单

2. **建立基础数据结构**
   - 供应商基本信息（名称、类型、历史供货数据）
   - 转运商基本信息（名称、历史损耗率数据）
   - 生产需求参数（产能、原材料消耗比例、成本系数）

### 核心算法设计

#### 算法1：最少供应商数量确定
```
输入：TOP50供应商数据、生产需求、安全库存要求
输出：最少供应商数量N

步骤：
1. 按供应商重要性排序（来自第一问结果）
2. 计算累计供应能力
3. 考虑供货不确定性，添加安全边际
4. 返回满足需求的最少供应商数量
```

#### 算法2：订购方案优化
```
输入：选定供应商集合、24周需求预测、成本参数
输出：最优订购计划矩阵

方法：多目标线性规划
- 目标1：最小化总成本
- 目标2：最大化供应稳定性
- 约束：需求满足、库存要求、供应能力限制
```

#### 算法3：转运方案优化
```
输入：订购方案、转运商损耗数据、运输能力
输出：最优转运分配方案

方法：匈牙利算法 + 启发式优化
- 构建成本矩阵（基于损耗率）
- 考虑运输能力约束
- 优化供应商-转运商匹配
```

### 关键技术点

#### 1. 不确定性处理
- **供货量预测**：基于历史数据的时间序列分析
- **需求波动**：考虑生产计划的不确定性
- **应急预案**：设计供应商备选方案

#### 2. 多目标优化
- **成本与风险平衡**：帕累托前沿分析
- **权重设定**：基于企业决策偏好
- **方案比较**：多准则决策分析（MCDM）

#### 3. 滚动优化策略
- **周期性重新规划**：每4周更新一次方案
- **反馈机制**：根据实际执行情况调整参数
- **学习能力**：历史数据不断更新模型参数

## 代码实现框架

### 主要模块划分
1. **数据加载模块**：`data_loader.py`
2. **需求预测模块**：`demand_forecast.py`  
3. **供应商选择模块**：`supplier_selection.py`
4. **订购优化模块**：`order_optimization.py`
5. **转运优化模块**：`transport_optimization.py`
6. **结果分析模块**：`result_analysis.py`
7. **可视化模块**：`visualization.py`

### 优化求解工具
- **线性规划**：PuLP库或Gurobi
- **整数规划**：用于0-1决策变量
- **启发式算法**：遗传算法（DEAP库）
- **统计分析**：SciPy、Statsmodels

### 输出结果格式
1. **订购方案表**：供应商 × 周次 矩阵
2. **转运方案表**：供应商-转运商分配表
3. **成本分析报告**：各项成本明细和趋势
4. **风险评估报告**：供应风险和应对措施
5. **可视化图表**：成本趋势、库存变化、供应商分布等

## 评估指标体系

### 经济性指标
- **总成本**：采购成本 + 运输成本 + 库存成本
- **单位成本**：每立方米产品的原材料成本
- **成本波动性**：成本标准差和变异系数

### 稳定性指标  
- **需求满足率**：实际满足需求/计划需求
- **库存充足率**：满足安全库存要求的周数比例
- **供应商集中度**：基尼系数或HHI指数

### 效率性指标
- **损耗率**：总损耗量/总供货量
- **运输利用率**：实际运输量/运输能力
- **库存周转率**：年消耗量/平均库存量

### 风险性指标
- **供应中断风险**：供应商失效概率
- **价格波动风险**：成本变异系数
- **需求冲击风险**：极端情况下的应对能力

## 具体实施步骤

### Step 1: 数据整合与预处理
```python
# 伪代码示例
def load_problem1_results():
    """加载第一问的分析结果"""
    top50_suppliers = pd.read_excel('TOP50最重要供应商.xlsx')
    reliability_data = pd.read_excel('供应商可靠性评估.xlsx') 
    transporter_analysis = pd.read_excel('转运商分析结果.xlsx')
    return top50_suppliers, reliability_data, transporter_analysis

def calculate_demand_parameters():
    """计算生产需求参数"""
    weekly_production = 28200  # 立方米/周
    material_ratios = {'A': 0.6, 'B': 0.66, 'C': 0.72}
    price_multipliers = {'A': 1.2, 'B': 1.1, 'C': 1.0}
    safety_stock_weeks = 2
    return weekly_production, material_ratios, price_multipliers, safety_stock_weeks
```

### Step 2: 最少供应商数量确定
```python
# 伪代码示例
def determine_minimum_suppliers(suppliers_data, demand_params):
    """确定最少供应商数量"""
    # 方法1: 确定性分析
    min_suppliers_deterministic = calculate_deterministic_minimum()
    
    # 方法2: 随机性分析（考虑供货不确定性）
    min_suppliers_stochastic = monte_carlo_simulation()
    
    # 方法3: 风险调整分析  
    min_suppliers_risk_adjusted = risk_adjusted_calculation()
    
    return max(min_suppliers_deterministic, min_suppliers_stochastic, min_suppliers_risk_adjusted)
```

### Step 3: 订购方案优化建模
```python
# 伪代码示例
def optimize_ordering_plan(selected_suppliers, weeks=24):
    """优化订购方案"""
    model = pulp.LpProblem("Material_Ordering", pulp.LpMinimize)
    
    # 决策变量：x[i][t] = 第t周向供应商i的订货量
    x = {}
    for supplier in selected_suppliers:
        for week in range(1, weeks+1):
            x[supplier, week] = pulp.LpVariable(f"Order_{supplier}_{week}", lowBound=0)
    
    # 目标函数：最小化总成本
    total_cost = calculate_total_cost_expression(x)
    model += total_cost
    
    # 约束条件
    add_demand_constraints(model, x)
    add_inventory_constraints(model, x)
    add_capacity_constraints(model, x)
    
    # 求解
    model.solve()
    return extract_solution(x)
```

### Step 4: 转运方案优化
```python
# 伪代码示例  
def optimize_transport_plan(ordering_plan, transporters_data):
    """优化转运方案"""
    transport_model = pulp.LpProblem("Transport_Assignment", pulp.LpMinimize)
    
    # 决策变量：y[i][j][t] = 第t周供应商i是否由转运商j运输
    y = {}
    for supplier in suppliers:
        for transporter in transporters:
            for week in range(1, 25):
                y[supplier, transporter, week] = pulp.LpVariable(
                    f"Transport_{supplier}_{transporter}_{week}", cat='Binary')
    
    # 目标函数：最小化总损耗
    total_loss = calculate_total_loss_expression(y)
    transport_model += total_loss
    
    # 约束条件
    add_assignment_constraints(transport_model, y)  # 每个供应商每周最多一个转运商
    add_capacity_constraints(transport_model, y)    # 转运商能力限制
    
    # 求解
    transport_model.solve()
    return extract_transport_solution(y)
```

### Step 5: 结果分析与验证
```python
# 伪代码示例
def analyze_solution_performance(ordering_plan, transport_plan):
    """分析方案性能"""
    # 经济性分析
    total_cost_analysis = calculate_cost_breakdown(ordering_plan)
    cost_comparison = compare_with_baseline(total_cost_analysis)
    
    # 稳定性分析  
    demand_satisfaction = simulate_demand_satisfaction(ordering_plan)
    inventory_analysis = analyze_inventory_levels(ordering_plan)
    
    # 风险分析
    supply_risk_assessment = assess_supply_risks(ordering_plan)
    sensitivity_analysis = perform_sensitivity_analysis()
    
    return {
        'cost_analysis': total_cost_analysis,
        'performance_metrics': demand_satisfaction,
        'risk_assessment': supply_risk_assessment
    }
```

## 预期结果与输出

### 1. 核心数值结果
- **最少供应商数量**：基于计算得出的具体数值（例如：35家）
- **24周订购计划表**：详细的订购量分配矩阵
- **转运分配方案**：供应商与转运商的最优匹配

### 2. 经济效益分析
- **总成本预估**：24周期间的总采购和运输成本
- **成本结构分析**：
  - 原材料采购成本占比
  - 运输成本占比  
  - 库存持有成本占比
- **与基准方案对比**：成本节约额度和节约率

### 3. 方案可行性验证
- **需求满足率**：≥ 95%（目标值）
- **库存安全性**：安全库存达成率 ≥ 90%
- **供应商利用率**：各供应商的使用频率分布
- **转运商负荷率**：运输能力利用效率

### 4. 风险控制效果
- **供应集中度**：避免过度依赖少数供应商
- **应急响应能力**：关键供应商断供时的应对方案
- **成本波动控制**：成本变异系数控制在合理范围

### 5. 可视化展示
- **订购量时间序列图**：展示24周的订购量变化趋势
- **供应商贡献度饼图**：各供应商供货量占比
- **转运商使用热力图**：转运商在不同时间的使用频率
- **成本构成堆积图**：各项成本的时间变化

### 6. 敏感性分析结果
- **需求变化影响**：±10%需求变化对方案的影响
- **供应商产能变化**：关键供应商产能下降的应对
- **转运成本变化**：运输费用波动的影响分析
